# Cleaning data
>This file contains functionality for writing data (timeseries and curreny-mappings) to the desired format to fasilitate further exploration and analysis. The *'data.xlsx'* file refers to the file where all the raw data gathered from datastream is stored.
>> Interest rates are gathered from Norges Bank

### Overview
* [Functonality](#Functionality)
* [Main Part](#Main-Part)
* [Equity Data](#Equity-Data)
* [Fundamental Pairs](#Fundamental-Pairs)
* [Valuta](#Valuta)
* [Factors](#Factors)
* [Indices](#Indices)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [117]:
### GLOBAL VARIABLES

PERIOD_START = '2013-01-02'

COUNTRIES_all = ['Norway', 'France', 'US', 'UK', 'Italy', 'Canada',
             'Australia', 'Austria', 'Hong Kong', 'Japan', 'Sweden', 'Chile',
             'Denmark', 'Finland', 'China']

COUNTRIES_for = ['France', 'Italy', 'Canada',
                 'Australia', 'Austria', 'Hong Kong', 'Japan', 'Sweden', 'Chile',
                 'Denmark', 'Finland', 'China']

COUNTRIES_for_2 = ['US', 'UK', 'Spain', 'Germany', 'Swiss']

Fundamental_NOR = pd.ExcelFile('FUNDAMENTAL_PAIRS_3.xlsx').sheet_names

indices2 = ['Germany', 'Spain']

## Functionality

In [3]:
def curr_mapping(filename: str, countries: list, skiprows:int = 0) -> dict:
    """
    Extracts the currencies of each of the equties in each country
    Return a nested dictionary, with countries at the top level,
    and equties at the second
    """
    
    all = {}
    
    for c in countries:
        
        try:
            data = pd.read_excel(filename,
                                 sheet_name=c,
                                 skiprows=skiprows,
                                 index_col='Name')
        except:
            print("An error occured when reading the file")
            
        single = {}
        
        for e in data.columns:
            try:
                curr = data[e].loc['CURRENCY']
                e_name = e[:-17]
                single[e_name] = curr
            except:
                print("An error occured when fetching the currencies")
            
        
        all[c] = single
        
        print(f'{c} currency')
        
        
    return all

        
def curr_mapping_country(countries: list, skiprows:int = 3) -> dict:
    
    curr = {}
    
    for c in countries:
        
        try:
            df = pd.read_excel('data.xlsx',
                               sheet_name=c,
                               skiprows=skiprows,
                               index_col='Name')
        except:
            print("An error occured when reading the file")
            
        curr[c] = df.iloc[0,0]
        
    return curr


def get_contry_eq_curr(country:  str,
                       filename: str,
                       skiprows: int = 3,
                       start:    str ='2013-01-01') -> (pd.DataFrame, str):
    """
    Reads and cleans data the equties of the specified country,
    and returns a dataframe with each equity as a sepreate column
    """
    try:
        df = pd.read_excel(filename,
                           sheet_name=country,
                           skiprows=skiprows,
#                            index_col='Name'
                          )
    except:
        print("An error occured when reading the file")
        
    curr = df.iloc[0,0]
    df = df.drop('CURRENCY')
    
    df.index = df.index.astype(str).str[:-9]
    assert type(df.index[0]) == str, "Wrong index type"
    
    df = df[start:]
    
    df.dropna(axis=1, inplace=True)
    assert df.isnull().sum().sum() == 0, "Null values still exist"

    
    return df, curr


def get_eqt_mult(contries: list,
                 filename: str,
                 skiprows: int = 0,
                 start:    str = '2013-01-01',
                 write_to_excel = False) -> dict:
    """
    Reads and cleans data for the equties for the specified countries.
    Returns a dictionary with the countries as keys, and the corrsponding
    dataframe of equities as values
    """
    
    d = {}
    
    for c in contries:
        data = pd.read_excel(filename,
                           sheet_name=c,
                           skiprows=skiprows,
                           index_col='Name')
        data = data.drop('CURRENCY')
        
        data.index = data.index.astype(str).str[:-9]
        assert type(data.index[0]) == str, "Wrong index type"
        
        column_names= [t[:-17] for t in data.columns]
        data.columns = column_names
        
        data = data[start:]
        
        data = data.astype(float)
        
        data.dropna(axis=1, inplace=True)
        assert data.isnull().sum().sum() == 0, "Null values still exist"
        
        
        if write_to_excel:
            data.to_excel(c + '.xlsx')
            print(f'{c} has been written to Excel')
        
        d[c] = data
        print(f'{c} successfully')
        
        

    return d

## Main Part

### Equity Data

In [5]:
### Reading the raw equity data
data_equity_all = get_eqt_mult(COUNTRIES_all,
                               filename='MASTER_A.xlsx',
                               start=PERIOD_START,
                               write_to_excel=False)
df_obx = data_equity_all.pop('Norway')

Norway successfully
France successfully
US successfully
UK successfully
Italy successfully
Canada successfully
Australia successfully
Austria successfully
Hong Kong successfully
Japan successfully
Sweden successfully
Chile successfully
Denmark successfully
Finland successfully
China successfully


In [73]:
### Reading the raw equity data
data_additional_eqt = get_eqt_mult(['NO_EQ_200', 'Germany', 'Spain', 'US', 'UK', 'Swiss'],
                               filename='MASTER_A2.xlsx',
                               start=PERIOD_START,
                               write_to_excel=False)

NO_EQ_200 successfully
Germany successfully
Spain successfully
US successfully
UK successfully
Swiss successfully


In [74]:
data_additional_eqt['Spain'].columns

Index(['BANCO SANTANDER', 'IBERDROLA', 'INDITEX', 'BBV.ARGENTARIA',
       'CAIXABANK', 'RED ELECTRICA', 'TELEFONICA', 'AMADEUS IT GROUP',
       'EDP RENOVAVEIS', 'ENAGAS',
       ...
       'TUBACEX', 'TUBOS REUNIDOS', 'UNION CATALANA DVL.',
       'URBAR INGENIEROS LTD DATA', 'URBAS GUADAHERMOSA',
       'VERTICE TRESCIENTOS SESENTA GRADOS', 'VIDRALA', 'VISCOFAN', 'VOCENTO',
       'ZARDOYA OTIS'],
      dtype='object', length=114)

### Fundamental Pairs

In [106]:
### Reading the raw equity data
fund_pairs = get_eqt_mult(Fundamental_NOR,
                               filename='FUNDAMENTAL_PAIRS_3.xlsx',
                               start=PERIOD_START,
                               write_to_excel=False)

Equinor successfully
DNB successfully
Telenor successfully
Hydro successfully
Aker BP successfully
Yara successfully
Gjensidige successfully
Orkla successfully
Schibsted A successfully
AF gruppen A successfully
Arendals fossekompani successfully
Atea successfully
Borregaard successfully
Frontline successfully
Kongsberg gruppen successfully
Nordic Semiconductor successfully
Subsea 7 successfully
Walenius Wilhelmsen successfully


In [107]:
fund_pairs['DNB'].head()

,NORDEA BANK,SVENSKA HANDELSBANKEN A,DANSKE BANK,SKANDINAVISKA ENSKILDA BANKEN A,BNP PARIBAS,LLOYDS BANKING GROUP,BARCLAYS
Name,,,,,,,
2013-01-02,1094.73,10414.08,12299.64,3502.65,395.47,63.08,28930.32
2013-01-03,1093.02,10475.52,12387.77,3493.39,395.47,63.10,28972.30
2013-01-04,1104.14,10594.02,12545.13,3539.68,394.80,63.30,29045.79
2013-01-07,1104.14,10611.57,12803.21,3564.37,402.14,64.11,30148.00
2013-01-08,1103.28,10589.63,13017.22,3579.80,406.45,64.57,30148.00


### Valuta

In [6]:
### Mapping currency
curr_mapping = curr_mapping('MASTER_A.xlsx', COUNTRIES_for)
unique_currencies = set(val for dic in curr_mapping.values() for val in dic.values())

France currency
US currency
UK currency
Italy currency
Canada currency
Australia currency
Austria currency
Hong Kong currency
Japan currency
Sweden currency
Chile currency
Denmark currency
Finland currency
China currency


In [115]:
curr_mapping

<function __main__.curr_mapping(filename: str, countries: list, skiprows: int = 0) -> dict>

### Valuta 2

In [75]:
### Mapping currency
curr_mapping_add = curr_mapping('MASTER_A2.xlsx', ['NO_EQ_200', 'Germany', 'Spain', 'US', 'UK', 'Swiss'])
unique_currencies_add = set(val for dic in curr_mapping_add.values() for val in dic.values())

NO_EQ_200 currency
Germany currency
Spain currency
US currency
UK currency
Swiss currency


In [76]:
set(curr_mapping_add['Swiss'].values())

{'E', 'SF', 'U$'}

In [99]:
curr_mapping_add

{'NO_EQ_200': {'EQUINOR': 'NK',
  'DNB BANK': 'NK',
  'MOWI': 'NK',
  'TELENOR': 'NK',
  'DNO': 'NK',
  'FRONTLINE': 'NK',
  'GAMING INNOVATION GROUP': 'NK',
  'GJENSIDIGE FORSIKRING': 'NK',
  'NORSK HYDRO': 'NK',
  'SCATEC': 'NK',
  'SCHIBSTED A': 'NK',
  'STOREBRAND': 'NK',
  'YARA INTERNATIONAL': 'NK',
  '5TH PLANET GAMES': 'NK',
  'ABG SUNDAL COLLIER HOLDING': 'NK',
  'AEGA': 'NK',
  "AF GRUPPEN 'A'": 'NK',
  'AKASTOR': 'NK',
  'AKER': 'NK',
  'AKER BP': 'NK',
  'AKER SOLUTIONS': 'NK',
  'AKVA GROUP': 'NK',
  'AQUALISBRAEMAR LOC': 'NK',
  'ARCHER': 'NK',
  'ARCTICZYMES TECHNOLOGIES': 'NK',
  'ARENDALS FOSSEKOMPANI': 'NK',
  'ASETEK': 'NK',
  'ATEA': 'NK',
  'AUSTEVOLL SEAFOOD': 'NK',
  'AVANCE GAS': 'NK',
  'AWILCO DRILLING': 'NK',
  'AWILCO LNG': 'NK',
  'AXACTOR': 'NK',
  'B2HOLDING': 'NK',
  'BAKKAFROST': 'NK',
  'BELSHIPS': 'NK',
  'BONHEUR': 'NK',
  "BORGESTAD 'A'": 'NK',
  'BORREGAARD': 'NK',
  'BOUVET': 'NK',
  'BW LPG': 'NK',
  'BW OFFSHORE': 'NK',
  'BYGGMA': 'NK',
  'CARA

### Valuta 3

In [114]:
curr_mapping_fund['Equinor']

{'BP': '£',
 'SHELL': 'E',
 'ENI': 'E',
 'TOTALENERGIES': 'E',
 'EXXON MOBIL': 'U$',
 'CHEVRON': 'U$'}

In [108]:
### Mapping currency
curr_mapping_fund = curr_mapping('FUNDAMENTAL_PAIRS_3.xlsx', fund_pairs)
unique_currencies_add = set(val for dic in curr_mapping_fund.values() for val in dic.values())

Equinor currency
DNB currency
Telenor currency
Hydro currency
Aker BP currency
Yara currency
Gjensidige currency
Orkla currency
Schibsted A currency
AF gruppen A currency
Arendals fossekompani currency
Atea currency
Borregaard currency
Frontline currency
Kongsberg gruppen currency
Nordic Semiconductor currency
Subsea 7 currency
Walenius Wilhelmsen currency


In [109]:
unique_currencies_add

{'C$', 'DK', 'E', 'I£', 'NK', 'SF', 'SK', 'TW', 'U$', '£'}

In [259]:
### Reading currency
df_forex = pd.read_excel('VALUTA_CLEAN.xlsx', index_col='Unnamed: 0'
                         )
df_forex.rename(columns={'EUR': 'E',
                         'USD': 'U$',
                         'GBP': '£',
                         'CAD': 'C$',
                         'AUD': 'A$',
                         'JPY': 'Y',
                         'SEK': 'SK',
                         'DKK': 'DK',
                         'RUB': 'UR',
                         'HKD': 'K$',
                         'CNY': 'CH',
                         'CHF': 'SF',
                         'TWD': 'TW',
                         'IDR': 'I£'
                         }, inplace=True)
df_forex = df_forex[PERIOD_START:]
df_forex.index = df_forex.index.astype(str)
df_forex = df_forex.reindex(fund_pairs['DNB'].index, axis='index')
df_forex = df_forex.fillna(method = 'ffill', limit=3)
df_forex.head()

,A$,BDT,BGN,BRL,BYN,C$,SF,CH,CZK,DK,...,SK,SGD,THB,TRY,TW,TWI,U$,VND,XDR,ZAR
Name,,,,,,,,,,,,,,,,,,,,,
2013-01-02,5.7905,NaN,3.7414,2.7032,NaN,5.6030,6.0525,0.8852,0.29017,0.9809,...,0.8538,4.5214,0.18186,3.1014,0.19021,90.22,5.5176,NaN,8.49825,0.6511
2013-01-03,5.8468,NaN,3.7281,2.7179,NaN,5.6423,6.0310,0.8924,0.28866,0.9774,...,0.8538,4.5518,0.18324,3.1222,0.19198,90.16,5.5652,NaN,8.53470,0.6481
2013-01-04,5.8420,NaN,3.7320,2.7445,NaN,5.6595,6.0362,0.9003,0.28787,0.9785,...,0.8556,4.5579,0.18385,3.1344,0.19318,90.30,5.6094,NaN,8.55374,0.6482
2013-01-07,5.8820,NaN,3.7356,2.7598,NaN,5.6763,6.0445,0.8993,0.28607,0.9794,...,0.8581,4.5563,0.18407,3.1439,0.19311,90.40,5.6032,NaN,8.55760,0.6519
2013-01-08,5.8764,NaN,3.7455,2.7661,NaN,5.6791,6.0581,0.8993,0.28630,0.9819,...,0.8541,4.5557,0.18390,3.1466,0.19296,90.44,5.5980,NaN,8.57660,0.6527


In [31]:
df_forex.index = pd.to_datetime(df_forex.index)

In [32]:
df_forex_weekly = df_forex.resample('W-Mon').sum()
df_forex_weekly

,A$,BDT,BGN,BRL,BYN,C$,CHF,CH,CZK,DK,...,SK,SGD,THB,TRY,TWD,TWI,U$,VND,XDR,ZAR
Name,,,,,,,,,,,,,,,,,,,,,
2013-01-07,23.3613,0.0000,14.9371,10.9254,0.0000,22.5811,24.1642,3.5772,1.15277,3.9162,...,3.4213,18.1874,0.73302,12.5019,0.76848,361.08,22.2954,0.000000,34.14429,2.5993
2013-01-14,29.3378,0.0000,18.7442,13.6837,0.0000,28.2282,30.2054,4.4721,1.43236,4.9135,...,4.2646,22.6856,0.91772,15.6834,0.96063,451.60,27.8290,0.000000,42.66202,3.2205
2013-01-21,29.3378,0.0000,18.9840,13.6544,0.0000,28.1884,29.9251,4.4807,1.45036,4.9752,...,4.2905,22.7341,0.93432,15.7887,0.96200,454.97,27.8668,0.000000,42.84539,3.1554
2013-01-28,29.0817,0.0000,18.9810,13.6126,0.0000,27.7263,29.8894,4.4587,1.44895,4.9747,...,4.2751,22.5503,0.92963,15.6815,0.95135,453.37,27.7419,0.000000,42.61503,3.0915
2013-02-04,28.5730,0.0000,18.9876,13.7893,0.0000,27.3844,30.0207,4.4042,1.44795,4.9778,...,4.3141,22.1452,0.92010,15.5556,0.92735,452.53,27.4193,0.000000,42.23074,3.0504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-21,32.0369,0.5185,25.8931,8.6506,17.3286,35.0868,48.4120,7.0375,2.07806,6.8065,...,4.7834,33.1624,1.38325,3.2721,1.60061,596.55,44.5955,0.001957,62.26029,2.9563
2022-02-28,32.1554,0.5179,25.6391,8.7498,16.0395,34.9492,48.3037,7.0522,2.02687,6.7398,...,4.7227,32.9806,1.37085,3.1997,1.59329,590.23,44.5638,0.001952,61.96244,2.9254
2022-03-07,25.6684,0.4112,20.1652,6.8472,11.8332,27.8564,38.4884,5.5972,1.54876,5.3028,...,3.6896,26.0324,1.08036,2.5360,1.25888,462.72,35.3336,0.001548,48.98164,2.2912


#### mapping currencies to equities

In [62]:
for country in data_equity_all.keys():
    for equity, currency in zip(data_equity_all[country], curr_mapping[country].values()):
        if currency in ['NK', 'CE']:
            continue

        data_equity_all[country][equity] =\
        data_equity_all[country][equity].mul(df_forex[currency])
        print(equity)
    print('***********************', country)

am = data_equity_all['US']

LVMH
L'OREAL
TOTALENERGIES
HERMES INTL.
SANOFI
CHRISTIAN DIOR
AIRBUS
BNP PARIBAS
KERING
SCHNEIDER ELECTRIC
ESSILORLUXOTTICA
L AIR LQE.SC.ANYME. POUR L ETUDE ET L EPXTN.
AXA
VINCI
DASSAULT SYSTEMES
PERNOD-RICARD
SAFRAN
CREDIT AGRICOLE
DANONE
ENGIE
SOCIETE GENERALE
ORANGE
STMICROELECTRONICS
CAPGEMINI
SARTORIUS STEDIM BIOTECH
SAINT GOBAIN
CMPG.DES ETS.MICH.
EDF
VEOLIA ENVIRON
LEGRAND
CARREFOUR
CNP ASSURANCES
EUROFINS SCIEN.
PUBLICIS GROUPE
RENAULT
TELEPERFORMANCE
THALES
ACCOR
ADP
BOLLORE
EIFFAGE
UNIBAIL RODAMCO WE STAPLED UNITS
ALSTOM
ARKEMA
BIOMERIEUX
BOUYGUES
BUREAU VERITAS
DASSAULT AVIATION
EDENRED
GECINA
IPSEN
REMY COINTREAU
REXEL
SCOR SE
SODEXO
VALEO
VIVENDI
AIR FRANCE-KLM
*********************** France
APPLE
MICROSOFT
AMAZON.COM
TESLA
ALPHABET A
NVIDIA
META PLATFORMS A
JP MORGAN CHASE & CO.
JOHNSON & JOHNSON
UNITEDHEALTH GROUP
PROCTER & GAMBLE
BANK OF AMERICA
WALMART
VISA 'A'
MASTERCARD
EXXON MOBIL
HOME DEPOT
BERKSHIRE HATHAWAY 'A'
COCA COLA
WALT DISNEY
PFIZER
CHEVRON
DYCOM INDS.
AB

PING AN BANK 'A'
S F HDG.'A'
WANHUA CHEMICAL GP.'A'
CITIC SECURITIES 'A'
EAST MONEY INFO.'A'
BANK OF NINGBO 'A'
INNER MONGOLIA YILI INDL.GP.'A'
JIANGSU HENGRUI MEDICINE 'A'
JIANGSU YANGHE BREW.JST. 'A'
SHAI.PUDONG DEV.BK. 'A'
BANK OF COMMS.'A'
BYD 'H'
CHINA STATE CON.ENGR.'A'
COSCO SHIPPING HDG.'A'
GREE ELECT.APP. 'A'
SAIC MOTOR 'A'
ZHANGZHOU PIENTZEHUANG PHARMS.'A'
ZIJIN MINING GROUP 'A'
AIER EYE HOSPITAL GP.'A'
BOE TECH.GP.'A'
CHINA PAC.IN.(GROUP) 'A'
CHINA VANKE 'A'
CHONGQING ZHIFEI BILG. PRDS.'A'
*********************** China


#### mapping currencies to equities 2

In [78]:
for country in data_additional_eqt.keys():
    for equity, currency in zip(data_additional_eqt[country], curr_mapping_add[country].values()):
        if currency in ['NK', 'CE']:
            continue

        data_additional_eqt[country][equity] =\
        data_additional_eqt[country][equity].mul(df_forex[currency])
        print(equity)
    print('***********************', country)

am = curr_mapping_add['US']

*********************** NO_EQ_200
MERCEDES-BENZ GROUP N
VOLKSWAGEN
ADIDAS
BMW
ALLIANZ
DEUTSCHE BANK
SAP
SIEMENS
BASF
BAYER
DEUTSCHE TELEKOM
E ON N
RWE
COMMERZBANK
DEUTSCHE LUFTHANSA
DEUTSCHE POST
HENKEL
BEIERSDORF
BILFINGER BERGER
BRENNTAG
CONTINENTAL
DEUTSCHE BOERSE
ENBW ENGE.BADEN-WURTG.
HANNOVER RUECK
INFINEON TECHNOLOGIES
MERCK KGAA
MTU AERO ENGINES HLDG.
MUENCHENER RUCK.
PORSCHE AML.HLDG.PREF.
PUMA
RHEINMETALL
SOFTWARE N
SYMRISE
THYSSENKRUPP
TUI
AAREAL BANK
ADESSO
ADIDAS (XET)
AIXTRON
ALSTRIA OFFICE REIT
ATOSS SOFTWARE
AURUBIS
BAYER (XET)
BECHTLE
BOSS (HUGO)
CANCOM
CARL ZEISS MEDITEC
DATAGROUP
ECKERT & ZIEGLER (XET)
EVOTEC
FRESENIUS
FRESENIUS MED.CARE
FUCHS PETROLUB N
GEA GROUP
GEA GROUP (XET)
GELSENWASSER
GERRY WEBER INTERNATIONAL
HAMB.HAFEN UD.LOGISTIK
HEIDELBERGCEMENT
HOCHTIEF
KUKA
LANXESS
NORCOM INFO.TECH.
PSI SOFTWARE
RHEINMETALL (XET)
SARTORIUS
SIXT
TELEFONICA DTL.HLDG.
UNITED INTERNET
WIRECARD
1&1
1&1 (XET)
11 88 0 SOLUTIONS
11 88 0 SOLUTIONS (XET)
2G ENERGY
***************

#### mapping currencies to equities 3

In [111]:
for eqt_NO in fund_pairs.keys():
    for equity, currency in zip(fund_pairs[eqt_NO], curr_mapping_fund[eqt_NO].values()):
        if currency in ['NK', 'CE']:
            continue

        fund_pairs[eqt_NO][equity] =\
        fund_pairs[eqt_NO][equity].mul(df_forex[currency])
        print(equity)
    print('***********************', eqt_NO)


BP
SHELL
ENI
TOTALENERGIES
EXXON MOBIL
CHEVRON
*********************** Equinor
NORDEA BANK
SVENSKA HANDELSBANKEN A
DANSKE BANK
SKANDINAVISKA ENSKILDA BANKEN A
BNP PARIBAS
LLOYDS BANKING GROUP
BARCLAYS
*********************** DNB
TELE2 B
TELIA COMPANY
DEUTSCHE TELEKOM
TELEFONICA
TELEKOM AUSTRIA
VODAFONE GROUP
SWISSCOM 'R'
BT GROUP
*********************** Telenor
KAISER ALUMINUM
*********************** Hydro
LUNDIN ENERGY
HARBOUR ENERGY
HESS
MARATHON OIL
MURPHY OIL
MEG ENERGY
*********************** Aker BP
CF INDUSTRIES HDG.
ICL GROUP
MOSAIC
*********************** Yara
TOPDANMARK
SAMPO 'A'
TRYG
ALLIANZ
ZURICH INSURANCE GROUP
ADMIRAL GROUP
TALANX AKTGSF.
*********************** Gjensidige
DANONE
NESTLE 'N'
SUEDZUCKER
TATE & LYLE
AAK
PREMIUM BRANDS HOLDINGS
UNILEVER (UK)
*********************** Orkla
SANOMA
INFORMA
LAGARDERE GROUPE
PEARSON
JCDECAUX
VIVENDI
*********************** Schibsted A
SKANSKA B
BALFOUR BEATTY
HOCHTIEF
NCC B
YIT
PEAB B
ACS ACTIV.CONSTR.Y SERV.
VINCI
BOSKALIS WESTMI

In [81]:
data_additional_eqt['US'].head()

,APPLE,DYCOM INDS.,MICROSOFT,AMAZON.COM,TESLA,PROSPERITY BCSH.,ALPHABET A,PARKER-HANNIFIN,NVIDIA,JP MORGAN CHASE & CO.,...,QUALCOMM,SALESFORCE.COM,SERVICENOW,SERVOTRONICS,T-MOBILE US,THERMO FISHER SCIENTIFIC,UNITED STATES STEEL,WORKDAY CLASS A,ABM INDS.,ACI WORLDWIDE
Name,,,,,,,,,,,,,,,,,,,,,
2013-01-02,93192.926112,74450.245848,206245.074024,72496.960272,816.659976,4837.831680,3977.306784,48610.552584,4305.879864,23177.947848,...,73724.019336,5490.949992,677.340576,915.093960,195.323040,27893.564688,897.106584,599.597592,9746.012760,2744.123184
2013-01-03,92809.727360,74061.904208,205237.619500,73454.963496,809.959208,4956.812336,4013.900500,49410.238636,4346.476852,23330.765352,...,74013.542620,5458.737724,671.663988,912.470192,195.616780,28047.606264,888.428528,597.758132,9887.913448,2782.766956
2013-01-04,90941.892654,75503.421504,202995.940182,74230.199892,807.697506,5065.007730,4125.769794,50073.262698,4525.551732,23932.897758,...,73503.670404,5531.429340,661.740918,940.696380,199.189794,28519.031010,905.357160,601.271586,10087.944960,2857.596642
2013-01-07,90306.998528,74938.149344,202392.402752,76811.691328,805.403968,4999.679328,4103.223360,49696.966048,4389.883072,23932.780064,...,74012.164512,5504.807808,623.187904,936.350752,195.495648,28709.788224,904.692672,600.382880,9960.304352,2856.287232
2013-01-08,90465.975180,74165.158980,201143.921220,76146.123240,789.206040,4983.787440,4091.298300,49690.199160,4290.195240,23957.984520,...,73827.935460,5531.943600,596.187000,937.832940,190.611900,28504.624140,879.221880,596.354940,9946.190520,2861.809560


#### Reading equities to disk

In [64]:
with pd.ExcelWriter('CLEANED_DATA.xlsx') as writer:
    for country in data_equity_all:
        data_equity_all[country].to_excel(writer, sheet_name=country)
        print(f'{country} written to disk')
    df_obx.to_excel(writer, sheet_name='Norway')

France written to disk
US written to disk
UK written to disk
Italy written to disk
Canada written to disk
Australia written to disk
Austria written to disk
Hong Kong written to disk
Japan written to disk
Sweden written to disk
Chile written to disk
Denmark written to disk
Finland written to disk
China written to disk


#### Reading equities to disk 2

In [82]:
with pd.ExcelWriter('CLEANED_DATA_2.xlsx') as writer:
    for country in data_additional_eqt:
        data_additional_eqt[country].to_excel(writer, sheet_name=country)
        print(f'{country} written to disk')
#     df_obx.to_excel(writer, sheet_name='Norway') fund_pairs

NO_EQ_200 written to disk
Germany written to disk
Spain written to disk
US written to disk
UK written to disk
Swiss written to disk


#### Reading equities to disk 2

In [112]:
with pd.ExcelWriter('FUND_PAIRS_CLEAN.xlsx') as writer:
    for EQT_NO in fund_pairs:
        fund_pairs[EQT_NO].to_excel(writer, sheet_name=EQT_NO)
        print(f'{EQT_NO} written to disk')
#     df_obx.to_excel(writer, sheet_name='Norway') fund_pairs

Equinor written to disk
DNB written to disk
Telenor written to disk
Hydro written to disk
Aker BP written to disk
Yara written to disk
Gjensidige written to disk
Orkla written to disk
Schibsted A written to disk
AF gruppen A written to disk
Arendals fossekompani written to disk
Atea written to disk
Borregaard written to disk
Frontline written to disk
Kongsberg gruppen written to disk
Nordic Semiconductor written to disk
Subsea 7 written to disk
Walenius Wilhelmsen written to disk


## Factors

#### Reading and formatting factors

In [275]:
factors_raw = pd.read_excel('MASTER_A.xlsx', sheet_name = 'Factors', index_col='Date')
factors_raw.drop('CURRENCY', axis=0, inplace=True)
factors_raw.drop(['Crude Oil BFO M1 Europe FOB $/BBl',
                  'S&P GSCI Silver Total Return - RETURN IND. (OFCL)'], axis=1, inplace=True)
factors_raw.index = factors_raw.index.astype(str).str[:-9]
factors_raw.columns = ['WTI Crude Oil', 'Gold Bullion', 'Brent Europe', 'Copper', 'Aluminium',
                       'Baltic Dry Index', 'Natural Gas - RFV', 'Natural Gas - Henry Hub',
                       'Fish Pool Spot', 'Silver', 'VLCC']
factors_raw = factors_raw.pct_change()[1:]
factors_raw.head()

,WTI Crude Oil,Gold Bullion,Brent Europe,Copper,Aluminium,Baltic Dry Index,Natural Gas - RFV,Natural Gas - Henry Hub,Fish Pool Spot,Silver,VLCC
Date,,,,,,,,,,,
2012-03-02,-0.018941,0.000198,0.001352,-0.005536,-0.011461,0.010485,0.025806,NaN,-0.038948,-0.004281,0.000000
2012-03-05,0.000187,-0.005743,0.005956,-0.008890,-0.018375,0.014267,0.002096,NaN,0.000000,-0.023502,0.017857
2012-03-06,-0.018928,-0.017639,-0.013025,-0.025232,-0.023064,0.006394,-0.014644,NaN,0.000000,-0.040211,-0.021930
2012-03-07,0.013945,0.006507,0.002719,0.000815,-0.012203,0.013977,0.006369,NaN,0.000000,0.020795,0.004484
2012-03-08,0.003956,0.006026,0.020659,0.004552,-0.001386,0.017544,0.021097,NaN,0.000000,0.000000,0.000000


#### Reading Bond reutrns

In [276]:
bond_returns = pd.read_excel('MASTER_A.xlsx',
                            skiprows=1,
                            sheet_name = 'Bond returns',
                            index_col='Date')
bond_returns = bond_returns['2012-03-02':]
bond_returns.index = bond_returns.index.astype(str)
bond_returns.reindex(factors_raw.index, axis='index')

assert (bond_returns.index == factors_raw.index).sum() == len(factors_raw), "indices do not match"

bond_returns.head()

,10Y NO,10Y US,10Y UK
Date,,,
2012-03-02,0.0425,-0.4335,-0.5525
2012-03-05,-0.0510,0.2380,0.2125
2012-03-06,-0.0255,-0.4845,-0.3825
2012-03-07,-0.0170,0.1870,-0.0085
2012-03-08,0.2720,0.3740,0.1190


#### Currency

In [278]:
currency = pd.read_excel('MASTER_A.xlsx',
                         sheet_name='Currency',
                         usecols=['NOK - USD', 'NOK - EUR', 'NOK - £', 'Date'],
                         index_col='Date',
                        )
currency = currency.pct_change()[1:]
currency = currency['2012-03-02':]
currency.index = currency.index.astype(str)
currency.head()

,NOK - USD,NOK - EUR,NOK - £
Date,,,
2012-03-02,0.007158,-0.002426,0.000662
2012-03-05,-0.003460,-0.001394,-0.002620
2012-03-06,0.017781,0.008783,0.009320
2012-03-07,-0.005078,-0.004694,-0.007100
2012-03-08,-0.016088,-0.004649,-0.009166


#### Concatenating the bond returns to the other factors

In [280]:
factors_returns_final = pd.concat([factors_raw, bond_returns, currency], axis=1)
factors_returns_final.tail()

,WTI Crude Oil,Gold Bullion,Brent Europe,Copper,Aluminium,Baltic Dry Index,Natural Gas - RFV,Natural Gas - Henry Hub,Fish Pool Spot,Silver,VLCC,10Y NO,10Y US,10Y UK,NOK - USD,NOK - EUR,NOK - £
Date,,,,,,,,,,,,,,,,,
2022-02-23,0.002067,0.004134,0.0,-0.005596,-0.006581,0.044693,0.110834,0.020000,0.0,0.009057,0.142496,0.2125,0.2465,0.0170,-0.001025,-0.001913,-0.003719
2022-02-24,0.007709,0.009465,0.0,0.001211,0.034327,-0.025401,0.530493,0.017429,0.0,0.008160,0.062731,-0.3485,-0.0425,-0.2380,0.023984,0.004953,0.002656
2022-02-25,-0.006680,-0.019584,0.0,-0.000050,-0.014119,-0.050754,-0.341122,-0.040685,0.0,-0.032376,0.199306,0.6120,0.1020,0.1105,-0.025575,-0.014042,-0.015225
2022-02-28,0.038290,0.008716,0.0,0.000101,0.000443,-0.017341,0.124291,-0.017857,0.0,0.013802,0.130670,-0.9690,-1.2325,-0.4165,-0.006463,-0.007661,-0.006467
2022-03-01,0.080234,0.011451,0.0,0.017441,0.033353,0.014216,0.164343,0.038636,0.0,0.037129,0.000000,-1.0030,-1.0880,-2.5670,0.006027,-0.002076,0.001119


#### Reading factors to disk

In [282]:
factors_returns_final.to_excel('FACTOR_RETURNS_VAL.xlsx', sheet_name='Factors')

### Indices

In [197]:
indices = pd.read_excel('MASTER_A.xlsx',
                            skiprows=2,
                            sheet_name = 'Indices',
                            index_col='Date')
keep_cols = [c.strip() for c in indices.columns if 'Unnamed' not in c]
indices = indices[keep_cols]
# indices.drop(['CHILE MSCI'], axis=1, inplace=True) # Dropping Chile -> what curreny is "CE" ??
indices.index = indices.index.astype(str)
indices.head()

,OSEBX,S&P 500,NASDAQ,DOW JONES,NYSE,FTSE 100,MSCI CANADA,AUSTRALIA,FRANCE CAC 40,France SBF 120,CSI300,MSCI CHINA,HANG SENG,TOPIX,Italy,STOCKHOLM OMX,COPENHAGEN OMX,HELSINKI OMX,Austria
Date,,,,,,,,,,,,,,,,,,,
2012-03-01,428.37,2367.87,3199.58,24203.45,10131.88,4038.79,5668.997,4894.136,7221.88,4974.01,2819.57,94.168,46193.19,1068.40,26503.71,113.70,532.63,11131.55,3418.70
2012-03-02,429.10,2360.28,3185.90,24198.37,10070.33,4025.09,5629.030,4912.968,7224.85,4978.89,2869.46,95.303,46572.09,1076.47,26616.96,113.76,532.23,11184.15,3415.19
2012-03-05,423.36,2351.28,3158.61,24170.85,10028.86,4000.36,5577.436,4908.118,7196.72,4956.85,2851.00,93.762,45930.71,1070.10,26435.30,113.00,530.43,11105.77,3370.97
2012-03-06,410.77,2315.21,3115.63,23795.12,9817.05,3926.13,5473.373,4842.502,6938.82,4782.21,2806.41,91.472,44939.20,1063.02,25539.13,109.00,517.50,10738.60,3235.60
2012-03-07,415.88,2331.89,3143.39,23946.75,9893.60,3951.14,5488.140,4775.471,7000.25,4827.68,2787.09,90.619,44553.73,1057.08,25823.88,109.58,520.46,10913.35,3259.80


In [198]:
### Currency mapping
ind_curr = dict(zip(keep_cols, range(len(keep_cols))))
ind_curr.pop('OSEBX')
ind_curr['S&P 500'] = 'U$'
ind_curr['NASDAQ'] = 'U$'
ind_curr['DOW JONES'] = 'U$'
ind_curr['NYSE'] = 'U$'
ind_curr['FTSE 100'] = '£'
ind_curr['AUSTRALIA'] = 'A$'
ind_curr['MSCI CANADA'] = 'C$'
ind_curr['FRANCE CAC 40'] = 'E'
ind_curr['France SBF 120'] = 'E'
# ind_curr['CHILE MSCI'] = ''
ind_curr['CSI300'] = 'CH'
# ind_curr['MSCI CHINA'] = ''
ind_curr['HANG SENG'] ='K$'
ind_curr['TOPIX'] = 'Y'
ind_curr['Italy'] ='E'
ind_curr['STOCKHOLM OMX'] = 'SK'
ind_curr['COPENHAGEN OMX'] = 'DK'
ind_curr['HELSINKI OMX'] = 'E'

ind_curr['Austria'] = 'E'
ind_curr.pop('MSCI CHINA')
final_map = ind_curr

final_map

{'S&P 500': 'U$',
 'NASDAQ': 'U$',
 'DOW JONES': 'U$',
 'NYSE': 'U$',
 'FTSE 100': '£',
 'MSCI CANADA': 'C$',
 'AUSTRALIA': 'A$',
 'FRANCE CAC 40': 'E',
 'France SBF 120': 'E',
 'CSI300': 'CH',
 'HANG SENG': 'K$',
 'TOPIX': 'Y',
 'Italy': 'E',
 'STOCKHOLM OMX': 'SK',
 'COPENHAGEN OMX': 'DK',
 'HELSINKI OMX': 'E',
 'Austria': 'E'}

In [ ]:
### Reindexing
indices = indices.reindex(df_obx.index, axis='index')
assert (indices.index == df_obx.index).sum() == len(df_obx), "Length mismatch"

### Getting the valuta right

for ind, curr in final_map.items():
    indices[ind] = indices[ind].mul(df_forex[curr])

indices_returns = indices.pct_change()[1:]


#### Reading indices to disk

In [199]:
indices_returns.to_excel('INDICES_RETURNS.xlsx', sheet_name='Indices')

In [207]:
indices.head()

,OSEBX,S&P 500,NASDAQ,DOW JONES,NYSE,FTSE 100,MSCI CANADA,AUSTRALIA,FRANCE CAC 40,France SBF 120,CSI300,MSCI CHINA,HANG SENG,TOPIX,Italy,STOCKHOLM OMX,COPENHAGEN OMX,HELSINKI OMX,Austria
Name,,,,,,,,,,,,,,,,,,,
2013-01-02,454.50,14172.231480,18593.760240,141156.650448,60383.400528,38094.958232,32171.927333,32852.285940,58880.483275,40628.150325,2441.178004,103.346,37165.785993,71.720906,202442.515775,102.481614,578.446539,83915.91920,28693.96640
2013-01-03,455.26,14264.664988,18683.823352,142149.399260,60733.695424,38237.753885,32243.171611,33392.945776,58470.830160,40400.159880,2461.033948,104.239,37623.523100,72.375680,201919.499175,102.549918,579.080178,84224.26233,28714.94781
2013-01-04,456.97,14447.907204,18838.889430,143747.550186,61641.977070,38472.655104,32517.534472,33273.280418,58670.237880,40533.755670,2484.260811,104.026,37807.571772,74.198656,202724.250750,103.270920,584.066650,84911.16474,28700.17893
2013-01-07,454.11,14386.944416,18800.809184,143044.541056,61357.785568,38378.307447,32481.020357,33458.221798,58326.866520,40355.859960,2492.877586,103.993,37768.821354,73.955372,202151.467440,103.083553,586.161106,84929.25454,28738.80854
2013-01-08,456.04,14331.831660,18741.096360,142398.565200,61086.271680,38279.207607,32518.566354,33228.785462,58502.102295,40456.026065,2482.400741,102.439,37378.760960,73.329568,203356.612550,102.244311,591.535836,84629.74338,28782.18252


In [209]:
indices_returns.head()

,OSEBX,S&P 500,NASDAQ,DOW JONES,NYSE,FTSE 100,MSCI CANADA,AUSTRALIA,FRANCE CAC 40,France SBF 120,CSI300,MSCI CHINA,HANG SENG,TOPIX,Italy,STOCKHOLM OMX,COPENHAGEN OMX,HELSINKI OMX,Austria
Name,,,,,,,,,,,,,,,,,,,
2013-01-03,0.001672,0.006522,0.004844,0.007033,0.005801,0.003748,0.002214,0.016457,-0.006957,-0.005612,0.008134,0.008641,0.012316,0.009129,-0.002584,0.000667,0.001095,0.003674,0.000731
2013-01-04,0.003756,0.012846,0.008299,0.011243,0.014955,0.006143,0.008509,-0.003584,0.003410,0.003307,0.009438,-0.002043,0.004892,0.025188,0.003986,0.007031,0.008611,0.008156,-0.000514
2013-01-07,-0.006259,-0.004219,-0.002021,-0.004891,-0.004610,-0.002452,-0.001123,0.005558,-0.005853,-0.004389,0.003469,-0.000317,-0.001025,-0.003279,-0.002825,-0.001814,0.003586,0.000213,0.001346
2013-01-08,0.004250,-0.003831,-0.003176,-0.004516,-0.004425,-0.002582,0.001156,-0.006857,0.003004,0.002482,-0.004203,-0.014943,-0.010328,-0.008462,0.005962,-0.008141,0.009169,-0.003527,0.001509
2013-01-09,0.004802,0.004897,0.006753,0.006852,0.005923,0.006726,0.002776,0.007683,0.003054,0.002678,0.002317,0.006013,0.006868,0.010084,0.022060,-0.000641,0.006122,-0.001633,0.005424


In [210]:
indices_returns.isnull().sum()

OSEBX             0
S&P 500           0
NASDAQ            0
DOW JONES         0
NYSE              0
FTSE 100          0
MSCI CANADA       0
AUSTRALIA         0
FRANCE CAC 40     0
France SBF 120    0
CSI300            0
MSCI CHINA        0
HANG SENG         0
TOPIX             0
Italy             0
STOCKHOLM OMX     0
COPENHAGEN OMX    0
HELSINKI OMX      0
Austria           0
dtype: int64

### Reading all currency mappings do disk

In [118]:
curr_mapping_1 = curr_mapping('MASTER_A.xlsx', COUNTRIES_for)

France currency
Italy currency
Canada currency
Australia currency
Austria currency
Hong Kong currency
Japan currency
Sweden currency
Chile currency
Denmark currency
Finland currency
China currency


In [119]:
curr_mapping_2 = curr_mapping('MASTER_A2.xlsx', COUNTRIES_for_2)

US currency
UK currency
Spain currency
Germany currency
Swiss currency


In [120]:
curr_mapping_3 = curr_mapping('FUNDAMENTAL_PAIRS_3.xlsx', Fundamental_NOR)

Equinor currency
DNB currency
Telenor currency
Hydro currency
Aker BP currency
Yara currency
Gjensidige currency
Orkla currency
Schibsted A currency
AF gruppen A currency
Arendals fossekompani currency
Atea currency
Borregaard currency
Frontline currency
Kongsberg gruppen currency
Nordic Semiconductor currency
Subsea 7 currency
Walenius Wilhelmsen currency


In [ ]:
COUNTRIES_for = ['France', 'Italy', 'Canada',
                 'Australia', 'Austria', 'Hong Kong', 'Japan', 'Sweden', 'Chile',
                 'Denmark', 'Finland', 'China']

COUNTRIES_for_2 = ['US', 'UK', 'Spain', 'Germany', 'Swiss']

In [255]:
MAP = {}
### curr_mapping_1
for country, eqt_dict_map in curr_mapping_1.items():
    MAP[country] = pd.DataFrame.from_dict(eqt_dict_map, orient='index')
    
### curr_mapping_2
for country, eqt_dict_map in curr_mapping_2.items():
    MAP[country] = pd.DataFrame.from_dict(eqt_dict_map, orient='index')
    
### curr_mapping_3
for eqt_nor, eqt_dict_map in curr_mapping_3.items():
    MAP[eqt_nor] = pd.DataFrame.from_dict(eqt_dict_map, orient='index')
    
MAP_FRAME = pd.concat(MAP)
MAP_FRAME = MAP_FRAME.droplevel(0, axis=0)

In [256]:
MAP_FRAME

,0
LVMH,E
L'OREAL,E
TOTALENERGIES,E
HERMES INTL.,E
SANOFI,E
...,...
HALLIBURTON,U$
SUBSEA 7,NK
WALLENIUS WILHELMSEN,NK
DFDS,DK


In [257]:
### Dropping duplicate index-names (multiple entries of same eqt)
MAP_FRAME_FINAL = MAP_FRAME[~MAP_FRAME.index.duplicated(keep='first')]
MAP_FRAME_FINAL

,0
LVMH,E
L'OREAL,E
TOTALENERGIES,E
HERMES INTL.,E
SANOFI,E
...,...
SCHLUMBERGER,U$
HALLIBURTON,U$
SUBSEA 7,NK
WALLENIUS WILHELMSEN,NK


In [258]:
MAP_FRAME_FINAL.to_excel('CURRENY_MAPPING_ALL_EQT.xlsx')

In [260]:
df_forex.to_excel('DF_FOREX.xlsx')